In [1]:
from bs4 import BeautifulSoup
%matplotlib inline
%config InlineBackend.figure_format='retina'
# import libraries
import numpy as np
import matplotlib as mp
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from importlib import reload
from datetime import datetime
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.cross_validation as cross_validation
import sklearn.cluster as cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from patsy import dmatrices

import seaborn as sns
import sys
import time
import os
import requests
import codecs
import json
import csv
import pandas as pd
import numpy as np
site_url = "https://www1.eeoc.gov/"
base_url = "https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1"

def get_page(url):
    response = requests.get(url)
    html = response.text.encode('utf-8')
    return html


In [2]:
def get_reports_page(url):
    url = url + "eeoc/statistics/employment/jobpat-eeo1"
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')
    all_reports = soup.find('div', {'class': 'CS_Textblock_Text'}).find('ul').find_all('a')
    for report in all_reports:
        year = report.text.split(" ")
        if year[0] != "Glossary":
#             if int(year[0]) == 2008:
#                 get_industry_page('https://www.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2008/nac2/index.html')
#             if int(year[0]) >= 2005 and int(year[0]) <= 2007:
#                 report_url = site_url + report['href']
#                 parse_report_page(report_url, year[0])  
            if int(year[0]) >= 2005 and int(year[0]) < 2007:
                report_url = site_url + report['href']
                parse_report_page(report_url, year[0])    

In [3]:
def parse_recent_report_page(url,year):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')    
    print(url)

In [4]:
def parse_report_page(url,year):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')    
    print(year)
    print("here")
    if soup.find('h1', text="This Page Was Not Found.") is None:
        links = soup.find_all('a', text='NAICS-2 Aggregate')
        for link in links:
            industry_url = base_url + "/"+  year  +"/" + link['href']
            print(industry_url)
            get_industry_page(industry_url)
    else:
        print('skip')

    

In [5]:
def get_industry_page(url):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')

    industries = soup.find('body').find('table').find_all('a',href=True)
    stripped_url = url.split('index.html')
    for industry in industries:
        industry_url = stripped_url[0] + industry['href']
        parse_industry(industry_url)

In [6]:
def parse_industry(url):
#     print(url)
    html = get_page(url)
    splitURl = (os.path.splitext(url)[0].split('/'))
    fileName = splitURl[7] +'-'+ splitURl[8]+'-'+splitURl[9]
    print(fileName)
    parseHTML(html,fileName)

    

In [7]:
def parseHTML(x,fileName):
#   This code parses the table that is available on EEOC website

    soup = BeautifulSoup((x),'html.parser')
    table = (soup.select('table'))[0]
    tableHeader = table.findAll('th',{'scope':'col'})

    headerRow = []
    for x in tableHeader:
        headerRow.append(x.text)
    
    body = (table.select('tbody'))
    temp=[]
    rows = []
    for h in body:
        row = (h.findAll('tr'))
        for r in row:
            temp.append((r.find('th').text))
            values = (r.select('td'))
            for v in values:
                temp.append(v.text)
            rows.append(temp)
            temp=[]

    with open(fileName, 'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow([headerRow[0],headerRow[1],headerRow[2],headerRow[3],headerRow[4],headerRow[5],headerRow[6],headerRow[7],headerRow[8],headerRow[9],headerRow[10]])
        for x in rows:
            if (len(x)==11):
                writer.writerow([x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9],x[10]])

In [8]:
get_reports_page(site_url)

2006
here
https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2006/nac2/index.html
2006-nac2-11
2006-nac2-21
2006-nac2-22
2006-nac2-23
2006-nac2-31
2006-nac2-32
2006-nac2-33
2006-nac2-42
2006-nac2-44
2006-nac2-45
2006-nac2-48
2006-nac2-49
2006-nac2-51
2006-nac2-52
2006-nac2-53
2006-nac2-54
2006-nac2-55
2006-nac2-56
2006-nac2-61
2006-nac2-62
2006-nac2-71
2006-nac2-72
2006-nac2-81
2006-nac2-92
2005
here
https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2005/nac2/index.html
2005-nac2-11
2005-nac2-21
2005-nac2-22
2005-nac2-23
2005-nac2-31
2005-nac2-32
2005-nac2-33
2005-nac2-42
2005-nac2-44
2005-nac2-45
2005-nac2-48
2005-nac2-49
2005-nac2-51
2005-nac2-52
2005-nac2-53
2005-nac2-54
2005-nac2-55
2005-nac2-56
2005-nac2-61
2005-nac2-62
2005-nac2-71
2005-nac2-72
2005-nac2-81
2005-nac2-92


In [11]:
def parseCSVfromWeb():
    rootdir = './'
    folders=[]
    for subdir, dirs, files in os.walk(rootdir):
        for d in dirs:
            if (d[0]=='2'):
                folders.append(d)
#         for f in files:
#             print (f)
#     print (folders)
    
    for f in folders:
        rootdir = './'+ f
#         print (rootdir)
        for subdir, dirs, files in os.walk(rootdir):
            for file in files:
                if (file[:2]=='20'):
                    print (file)
                    f = rootdir + '/' +file
                    getDFfromMyCSV(f)
        #         for d in subdzr:
#             print (d)
#         year = (subdir.split('-')[0][2:])
parseCSVfromWeb()

2005 national agg scraped from web.csv
here
./2005-Data/2005 national agg scraped from web.csv
2005 national agg.csv
here
./2005-Data/2005 national agg.csv
2005-nac2
here
./2005-Data/2005-nac2
2005-nac2-11
here
./2005-Data/2005-nac2-11
2005-nac2-21
here
./2005-Data/2005-nac2-21
2005-nac2-22
here
./2005-Data/2005-nac2-22
2005-nac2-23
here
./2005-Data/2005-nac2-23
2005-nac2-31
here
./2005-Data/2005-nac2-31
2005-nac2-32
here
./2005-Data/2005-nac2-32
2005-nac2-33
here
./2005-Data/2005-nac2-33
2005-nac2-42
here
./2005-Data/2005-nac2-42
2005-nac2-44
here
./2005-Data/2005-nac2-44
2005-nac2-45
here
./2005-Data/2005-nac2-45
2005-nac2-48
here
./2005-Data/2005-nac2-48
2005-nac2-49
here
./2005-Data/2005-nac2-49
2005-nac2-51
here
./2005-Data/2005-nac2-51
2005-nac2-52
here
./2005-Data/2005-nac2-52
2005-nac2-53
here
./2005-Data/2005-nac2-53
2005-nac2-54
here
./2005-Data/2005-nac2-54
2005-nac2-55
here
./2005-Data/2005-nac2-55
2005-nac2-56
here
./2005-Data/2005-nac2-56
2005-nac2-61
here
./2005-Data/200

CParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3


In [9]:
def getDFfromMyCSV(fileName):
#   this code creates a data frame from the CSV that we created 

    print ('here')
    print (fileName)
    df = pd.DataFrame.from_csv(fileName)
    return (df)
#     print (df.drop(df.index[[22,23,24,25,26,27,28]]))
# getDFfromCSV()

In [12]:
def getDFfromGOVCSV(csv_input, file):
#   this code creates a data frame from the CSV file that the government provides

    index = ['ALL EMPLOYEES','Men','Women','WHITE','White Men','White Women','MINORITY',
             'Minority Men','Minority Women','BLACK','Black Men','Black Women','HISPANIC','Hispanic Men','Hispanic Women',
             'ASIAN AMERICAN','Asian Men', 'Asian Women','AMERICAN INDIAN','American-Indian Men','American-Indian Women',
             'HAWAIIAN','Hawaiian Men','Hawaiian Women','TWO OR MORE RACES','Multiracial Men','Multiracial Women']
    column = ['Total Employment','Officials & Managers','Professionals','Technicians',
              "Sales Workers",'Office & Clerical Workers','Craft Workers','Operatives',
              'Laborers','Service Workers']
#     myDF = myDF.fillna(0)

    if file:
        dfFromFile = pd.DataFrame.from_csv(csv_input,sep=';')
    else:
        dfFromFile = csv_input
            
    sAndRace = ['TOTAL','MT','FT','WHT','WHM','WHF','MinT','MinM','MinF','BLKT','BLKM','BLKF','HISPT',
                      'HISPM','HISPF','ASIANT','ASIANM','ASIANF','AIANT','AIANM','AIANF',
                      'nhopiT','NHOPIM','NHOPIF','tomrT','TOMRM','TOMRF']
    table = []
    for x in sAndRace:
        tempRow = []
        for y in range(1,11):
            if (y==1):
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x +'1')))
            elif (y==2):
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x+'1_2')))
            elif (y==10):
                tempRow.insert(0,dfFromFile.get_value(index='UNITED STATES',col=(x+'10')))
            else:
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x+str(y))))
#             print (tempRow)
        table.append(tempRow)
    

    myDF = pd.DataFrame(data=table, index=index, columns=column)
    if not file:
        print(dfFromFile['NAC2_Label'][0])
    
    return(myDF)


In [12]:
from sklearn import preprocessing

def linear_regression(xDF, yDF):
    
    X, y = utils.shuffle(xDF, yDF, random_state=1)
    X_train, X, y_train, y = cross_validation.train_test_split(
    X, y, test_size=0.4, random_state=0)
    print((X_train.shape), y_train.shape)
    print((X.shape), y.shape)
    
        
#    fig, axes = plt.subplots(2,2,figsize=(15,10))

    for i in range(4):
#         plt_i = i // 2
#         plt_j = i % 2
        

        subX_train = X_train[X_train.columns[i]]
    
        
        le = preprocessing.LabelEncoder()
        subX_train = le.fit_transform(subX_train)
        
        X_train[X_train.columns[i]]= subX_train

#         axes[plt_i][plt_j].scatter(subX_train, y_train, c="slategray", alpha=0.4, linewidths=0.3)
#         axes[plt_i][plt_j].set_xlabel(X_train.columns[i])
#         axes[plt_i][plt_j].set_ylabel('Average'); 
        

    model = sm.OLS(y_train, X_train)
    results = model.fit()
    print(results.summary())

In [13]:
#gets dataframes from csv files and puts them in a list and accesses each job type(Sarah's files) - still need to do NAC classifications
#COMPLETE DF
dfList = []
# df05 = getDFfromMyCSV('2005 National agg.csv')
# dfList.append(df05) 

# df06 = getDFfromMyCSV('2006 National agg.csv')
# dfList.append(df06) 

df07 = getDFfromGOVCSV('YEAR07_US.txt',True)
dfList.append(df07) 

df08 = getDFfromGOVCSV('YEAR08_US.txt',True)
dfList.append(df08)

df09 = getDFfromGOVCSV('YEAR09_US.txt',True)
dfList.append(df09)

df10 = getDFfromGOVCSV('YEAR10_US.txt',True)
dfList.append(df10)

df11 = getDFfromGOVCSV('year11_us.txt',True)
dfList.append(df11)

df12 = getDFfromGOVCSV('year12_us.txt',True)
dfList.append(df12)

df13 = getDFfromGOVCSV('year13_us.txt',True)
dfList.append(df13)

df14 = getDFfromGOVCSV('year14_us.txt',True)
dfList.append(df14)

df15 = getDFfromGOVCSV('year15_us.txt',True)
dfList.append(df15)


graphGenderRace = ['White Men','White Women','Minority Men','Minority Women',
          'Black Men','Black Women','Hispanic Men','Hispanic Women',
          'Asian Men', 'Asian Women','American-Indian Men','American-Indian Women',
          'Hawaiian Men','Hawaiian Women','Multiracial Men','Multiracial Women']

columnX = ['Race', 'Gender','Year', 'JobType', 'Workers']   

jobTypes = ['Officials & Managers', 'Professionals','Technicians', 'Sales Workers', 'Office & Clerical Workers',
'Craft Workers','Operatives','Laborers', 'Service Workers']

OfficialsManagers = [] 
Professionals = [] 
Technicians = []
SalesWorkers = []
OfficeClericalWorkers = []
CraftWorkers = []
Operatives = []
Laborers = [] 
ServiceWorkers = []

year = 2007 #TODO not sure if year goes up or down
for df in dfList:
    for jobType in jobTypes:
        for demographic in graphGenderRace:

            raceGender = demographic.split()
            
            if jobType == 'Officials & Managers':
                tempX = OfficialsManagers
            if jobType == 'Professionals':
                tempX = Professionals
            if jobType == 'Technicians':
                tempX = Technicians
            if jobType == 'Sales Workers':
                tempX = SalesWorkers
            if jobType == 'Office & Clerical Workers':
                tempX = OfficeClericalWorkers
            if jobType == 'Craft Workers':
                tempX = CraftWorkers
            if jobType == 'Operatives':
                tempX = Operatives
            if jobType == 'Laborers':
                tempX = Laborers
            if jobType == 'Service Workers':
                tempX = ServiceWorkers
                
            tempX.append([raceGender[0],raceGender[1],year,jobType,df[jobType][demographic]])

    year += 1

OfficialsManagersDF = pd.DataFrame(data=OfficialsManagers, columns=columnX)
ProfessionalsDF = pd.DataFrame(data=Professionals, columns=columnX)
TechniciansDF = pd.DataFrame(data=Technicians, columns=columnX)
SalesWorkersDF = pd.DataFrame(data=SalesWorkers, columns=columnX)
OfficeClericalWorkersDF = pd.DataFrame(data=OfficeClericalWorkers, columns=columnX)
CraftWorkersDF = pd.DataFrame(data=CraftWorkers, columns=columnX)
OperativesDF = pd.DataFrame(data=Operatives, columns=columnX)
LaborersDF = pd.DataFrame(data=Laborers, columns=columnX)
ServiceWorkersDF = pd.DataFrame(data=ServiceWorkers, columns=columnX)
allData = OfficialsManagers + Professionals + Technicians + SalesWorkers + OfficeClericalWorkers + CraftWorkers + Operatives + Laborers + ServiceWorkers


allWorkers = pd.DataFrame(data=allData, columns=columnX)

print(allWorkers)
print(allWorkersY)

OSError: File b'YEAR07_US.txt' does not exist

In [20]:
#gets dataframes from csv files and puts them in a list and accesses each job type(Sarah's files) - still need to do NAC classifications
dfList = []
# df05 = getDFfromMyCSV('2005 National agg.csv')
# dfList.append(df05) 

# df06 = getDFfromMyCSV('2006 National agg.csv')
# dfList.append(df06) 

df07 = getDFfromGOVCSV('YEAR07_US.txt',True)
dfList.append(df07) 

df08 = getDFfromGOVCSV('YEAR08_US.txt',True)
dfList.append(df08)

df09 = getDFfromGOVCSV('YEAR09_US.txt',True)
dfList.append(df09)

df10 = getDFfromGOVCSV('YEAR10_US.txt',True)
dfList.append(df10)

df11 = getDFfromGOVCSV('year11_us.txt',True)
dfList.append(df11)

df12 = getDFfromGOVCSV('year12_us.txt',True)
dfList.append(df12)

df13 = getDFfromGOVCSV('year13_us.txt',True)
dfList.append(df13)

df14 = getDFfromGOVCSV('year14_us.txt',True)
dfList.append(df14)

df15 = getDFfromGOVCSV('year15_us.txt',True)
dfList.append(df15)


graphGenderRace = ['White Men','White Women','Minority Men','Minority Women',
          'Black Men','Black Women','Hispanic Men','Hispanic Women',
          'Asian Men', 'Asian Women','American-Indian Men','American-Indian Women',
          'Hawaiian Men','Hawaiian Women','Multiracial Men','Multiracial Women']

columnX = ['Race', 'Gender','Year', 'JobType']   
columnY = ['Workers']

jobTypes = ['Officials & Managers', 'Professionals','Technicians', 'Sales Workers', 'Office & Clerical Workers',
'Craft Workers','Operatives','Laborers', 'Service Workers']

OfficialsManagers = [] 
Professionals = [] 
Technicians = []
SalesWorkers = []
OfficeClericalWorkers = []
CraftWorkers = []
Operatives = []
Laborers = [] 
ServiceWorkers = []

OfficialsManagersY = [] 
ProfessionalsY = [] 
TechniciansY = []
SalesWorkersY= []
OfficeClericalWorkersY = []
CraftWorkersY = []
OperativesY = []
LaborersY = [] 
ServiceWorkersY = []

year = 2007 #TODO not sure if year goes up or down
for df in dfList:
    for jobType in jobTypes:
        for demographic in graphGenderRace:

            raceGender = demographic.split()
            
            
            if jobType == 'Officials & Managers':
                tempX = OfficialsManagers
                tempY =OfficialsManagersY
            if jobType == 'Professionals':
                tempX = Professionals
                tempY =ProfessionalsY
            if jobType == 'Technicians':
                tempX = Technicians
                tempY =TechniciansY
            if jobType == 'Sales Workers':
                tempX = SalesWorkers
                tempY = SalesWorkersY
            if jobType == 'Office & Clerical Workers':
                tempX = OfficeClericalWorkers
                tempY = OfficeClericalWorkersY
            if jobType == 'Craft Workers':
                tempX = CraftWorkers
                tempY = CraftWorkersY
            if jobType == 'Operatives':
                tempX = Operatives
                tempY = OperativesY
            if jobType == 'Laborers':
                tempX = Laborers
                tempY = LaborersY
            if jobType == 'Service Workers':
                tempX = ServiceWorkers
                tempY = ServiceWorkersY
                
            tempX.append([raceGender[0],raceGender[1],year,jobType])
            tempY.append([df[jobType][demographic]])

    year += 1

OfficialsManagersDF = pd.DataFrame(data=OfficialsManagers, columns=columnX)
OfficialsManagersDFY = pd.DataFrame(data=OfficialsManagersY, columns=columnY)

ProfessionalsDF = pd.DataFrame(data=Professionals, columns=columnX)
ProfessionalsDFY = pd.DataFrame(data=ProfessionalsY, columns=columnY)

TechniciansDF = pd.DataFrame(data=Technicians, columns=columnX)
TechniciansDFY = pd.DataFrame(data=TechniciansY, columns=columnY)

SalesWorkersDF = pd.DataFrame(data=SalesWorkers, columns=columnX)
SalesWorkersDFY = pd.DataFrame(data=SalesWorkersY, columns=columnY)

OfficeClericalWorkersDF = pd.DataFrame(data=OfficeClericalWorkers, columns=columnX)
OfficeClericalWorkersDFY = pd.DataFrame(data=OfficeClericalWorkersY, columns=columnY)

CraftWorkersDF = pd.DataFrame(data=CraftWorkers, columns=columnX)
CraftWorkersDFY = pd.DataFrame(data=CraftWorkersY, columns=columnY)

OperativesDF = pd.DataFrame(data=Operatives, columns=columnX)
OperativesDFY = pd.DataFrame(data=OperativesY, columns=columnY)

LaborersDF = pd.DataFrame(data=Laborers, columns=columnX)
LaborersDFY = pd.DataFrame(data=LaborersY, columns=columnY)

ServiceWorkersDF = pd.DataFrame(data=ServiceWorkers, columns=columnX)
ServiceWorkersDFY = pd.DataFrame(data=ServiceWorkersY, columns=columnY)

allData = OfficialsManagers + Professionals + Technicians + SalesWorkers + OfficeClericalWorkers + CraftWorkers + Operatives + Laborers + ServiceWorkers
allDataY = OfficialsManagersY + ProfessionalsY + TechniciansY + SalesWorkersY + OfficeClericalWorkersY + CraftWorkersY + OperativesY + LaborersY + ServiceWorkersY


allWorkers = pd.DataFrame(data=allData, columns=columnX)
allWorkersY = pd.DataFrame(data=allDataY, columns=columnY)

print(allWorkers)
print(allWorkersY)




# linear_regression(allWorkers, allWorkersY)

# print("Technical ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(TechniciansDF, TechniciansDFY)

# print("Clerical Workers ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(OfficeClericalWorkersDF, OfficeClericalWorkersDFY)

# print("Laborers ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(LaborersDF, LaborersDFY)

# print("Professional +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(ProfessionalsDF, ProfessionalsDFY)

# print("Manager +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(OfficialsManagersDF, OfficialsManagersDFY)

# print("Service +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(ServiceWorkersDF, ServiceWorkersDFY)

# print("Sales +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(SalesWorkersDF, SalesWorkersDFY)

# print("Craft +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(CraftWorkersDF, CraftWorkersDFY)

# print("Operative +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print()
# linear_regression(OperativesDF, OperativesDFY)

                 Race Gender  Year               JobType
0               White    Men  2007  Officials & Managers
1               White  Women  2007  Officials & Managers
2            Minority    Men  2007  Officials & Managers
3            Minority  Women  2007  Officials & Managers
4               Black    Men  2007  Officials & Managers
5               Black  Women  2007  Officials & Managers
6            Hispanic    Men  2007  Officials & Managers
7            Hispanic  Women  2007  Officials & Managers
8               Asian    Men  2007  Officials & Managers
9               Asian  Women  2007  Officials & Managers
10    American-Indian    Men  2007  Officials & Managers
11    American-Indian  Women  2007  Officials & Managers
12           Hawaiian    Men  2007  Officials & Managers
13           Hawaiian  Women  2007  Officials & Managers
14        Multiracial    Men  2007  Officials & Managers
15        Multiracial  Women  2007  Officials & Managers
16              White    Men  2

In [21]:
def get_NAC_DFfromGOVCSV(fileName):
    dfFromFile = pd.DataFrame.from_csv(fileName,sep=';')

    keys = list(dfFromFile.keys())
    
    for row in dfFromFile.itertuples():
        newDF = pd.DataFrame(data=[row[1:]],index=['UNITED STATES'], columns=keys)        
        getDFfromGOVCSV(newDF, False)

# get_NAC_DFfromGOVCSV('year15_nac2.txt')
# get_NAC_DFfromGOVCSV('year14_nac2.txt')
# get_NAC_DFfromGOVCSV('year13_nac2.txt')
# get_NAC_DFfromGOVCSV('year12_nac2.txt')
# get_NAC_DFfromGOVCSV('year11_nac2.txt')
# get_NAC_DFfromGOVCSV('year10_nac2.txt')
#get_NAC_DFfromGOVCSV('year09_nac2.txt')
#get_NAC_DFfromGOVCSV('year08_nac2.txt')
#get_NAC_DFfromGOVCSV('year07_nac2.txt')




    